In [1]:
'''
# 라이브러리 정의
 - 시스템 라이브러리
 - 오디오 관련 라이브러리
 - 연산 및 Plot 관련 라이브러리 
 - 샤용자 라이브러리
'''

# Import Systems 
import struct
import io
import os
import sys
import math
import platform
import time

# Import Audio
import pyaudio
import sounddevice
import librosa
import soundfile

import numpy as np
import scipy
import scipy.signal as sig
import matplotlib.pyplot as plt
from tkinter import TclError

# User Libraries
import pyOssWavfile
import pyRoomAcoustic as room
import pyOssDebug as dbg
import pyOssFilter
import pyOssPyAudio

%matplotlib tk


In [3]:
set_device_type = 'asio'

if 'darwin' in platform.system():
    IDX = pyaudio.paCoreAudio           # = 5
elif 'linux' in platform.system():
    IDX = pyaudio.paALSA
else:
    if set_device_type == 'asio':
        IDX = pyaudio.paASIO            # = 3
    elif set_device_type == 'wasapi':
        IDX = pyaudio.paWASAPI          # = 13
    elif set_device_type == 'wdm':
        IDX = pyaudio.paWDMKS           # = 11
    else:
        IDX = pyaudio.paMME             # = 2

print('Index of Device Type:', IDX)

# List of Device Names IDX 
dev_name = pyOssPyAudio.ju_get_device_name(IDX)
print(dev_name)


Index of Device Type: 3
['ASIO4ALL v2', 'Yamaha Steinberg USB ASIO']


# 장치선택: Windows MME

tgt_dev_in = dev_name[0]        # Microsoft 사운드 매퍼 - Input
tgt_dev_out = dev_name[2]       # Microsoft 사운드 매퍼 - Output

dev_in = pyOssPyAudio.ju_get_device_info(IDX, tgt_dev_in)
st_dev_in = pyOssPyAudio.CAudioDeviceInfo()

st_dev_in.index = dev_in['index']
st_dev_in.name = dev_in['name']
st_dev_in.maxInCh = dev_in['maxInputChannels']
st_dev_in.maxOutCh = dev_in['maxOutputChannels']
st_dev_in.fs = int(dev_in['defaultSampleRate'])

# dev_in_index = dev_in['index']
# dev_in_name = dev_in['name']
# dev_in_ch = dev_in['maxInputChannels']
# dev_in_fs = int(dev_in['defaultSampleRate'])

dev_out = pyOssPyAudio.ju_get_device_info(IDX, tgt_dev_out)
st_dev_out = pyOssPyAudio.CAudioDeviceInfo()

st_dev_out.index = dev_out['index']
st_dev_out.name = dev_out['name']
st_dev_out.maxInCh = dev_out['maxInputChannels']
st_dev_out.maxOutCh = dev_out['maxOutputChannels']
st_dev_out.fs = int(dev_out['defaultSampleRate'])


In [4]:
# 장치 선택: ASIO
# tgt_asio_dev_name = 'ASIO4ALL v2'
tgt_asio_dev_name = 'Yamaha Steinberg USB ASIO'
# tgt_asio_dev_name = 'Focusrite USB ASIO'

dev_asio = pyOssPyAudio.ju_get_device_info(IDX, tgt_asio_dev_name)
st_dev_asio = pyOssPyAudio.CAudioDeviceInfo(dev_asio)

print('dev_asio: ', dev_asio)
print(st_dev_asio.index, st_dev_asio.name, st_dev_asio.maxInCh, st_dev_asio.maxOutCh, st_dev_asio.fs)


dev_asio:  {'index': 17, 'structVersion': 2, 'name': 'Yamaha Steinberg USB ASIO', 'hostApi': 2, 'maxInputChannels': 6, 'maxOutputChannels': 6, 'defaultLowInputLatency': 0.011609977324263039, 'defaultLowOutputLatency': 0.011609977324263039, 'defaultHighInputLatency': 0.011609977324263039, 'defaultHighOutputLatency': 0.011609977324263039, 'defaultSampleRate': 44100.0}
17 Yamaha Steinberg USB ASIO 6 6 44100


In [ ]:
# Record & save audio data from Mic using ASIO Device

pa = pyaudio.PyAudio()

FORMAT = pa.get_format_from_width(2)
CH = st_dev_asio.maxInCh
# CH = st_dev_in.maxInCh
# CH = 1
# CH = 2
CHUNK = 1024 * 2
RATE = st_dev_asio.fs            # device 정보에서 읽어오면 됨
BUFFER = CHUNK * 2 * CH
RECORD_SECONDS = 10

print(pa.is_format_supported(RATE, 
            # input_device=dev_asio_index,
            input_device = st_dev_asio.index,
            input_channels=CH,
            input_format=FORMAT,
     ))

stream1 = pa.open(
    format = FORMAT,
    channels = CH,
    rate = RATE,
    input = True,
    output = False,
    frames_per_buffer = BUFFER,
    input_device_index = st_dev_asio.index
    )

# stream2 = pa.open(
#     format = FORMAT,
#     channels = 2,
#     rate = RATE,
#     input = True,
#     output = False,
#     frames_per_buffer = BUFFER,
#     input_device_index = dev_asio_index
# )


In [ ]:
# create matplotlib figure and axes
fig, ax = plt.subplots(1, figsize=(15, 7))

# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)

# create a line object with random data
line, = ax.plot(x, np.random.rand(CHUNK), '-', lw=2)

# basic formatting for the axes
ax.set_title('AUDIO WAVEFORM')
ax.set_xlabel('samples')
ax.set_ylabel('volume')
# ax.set_ylim(0, 256)
ax.set_ylim(-128, 128)
ax.set_xlim(0, 2 * CHUNK)
# plt.setp(ax, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])
plt.setp(ax, xticks=[0, CHUNK, 2 * CHUNK], yticks=[-128, 0, 128])

# show the plot
plt.show(block=False)

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()
sel_ch = 1

while True:
    # binary data
    # data = stream1.read(CHUNK, exception_on_overflow=False)
    data = np.fromstring(stream1.read(CHUNK, exception_on_overflow=False), dtype=np.int16)
    # convert data to integers, make np array, then offset it by 127
    # data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    # data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset by 128
    # data_np = np.array(data_int, dtype='b')[::2] + 128
    chunk_length = np.int(len(data) / CH)
    data_temp = np.reshape(data, (chunk_length, CH))
    data_np = data_temp[:,sel_ch]

    line.set_ydata(data_np)
    
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break



# Record input audio data
print("* recording")

frames1 = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data1 = stream1.read(CHUNK)
    frames1.append(data1)

print("* done recording")

In [ ]:
# Stream Stop Process 

stream1.stop_stream()
stream1.close()
pa.terminate()          #


# Not really sure what b'' means in BYTE STRING but numpy needs it 
# just like wave did...
framesAll = b''.join(frames1)

# Use numpy to format data and reshape.  
# PyAudio output from stream.read() is interlaced.
result = np.fromstring(framesAll, dtype=np.int16)
chunk_length = np.int(len(result) / CH)
result = np.reshape(result, (chunk_length, CH))


# Write multi-channel .wav file with SciPy
pyOssWavfile.write('pyaudio_rec.wav', RATE, result)
print("* done save wav file")